In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import etree

string = input("Que quieres buscar?")
url = 'https://listado.mercadolibre.com.ar/()#D[A:{}]'.format(string.replace(' ', '-'), string)
response = requests.get(url)

print(response.status_code)

response.text

soup = BeautifulSoup(response.content, 'html.parser')

titulos = soup.find_all('h2', attrs={"class":"ui-search-item__title"})
titulos = [i.text for i in titulos]

urls = soup.find_all('a', attrs={"class":"ui-search-item__group__element ui-search-link__title-card ui-search-link"})
urls = [i.get('href') for i in urls]

dom = etree.HTML(str(soup))
precios = dom.xpath('//li[contains(@class, "ui-search-layout__item")]//span[@class="andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript"]/span[@class="andes-money-amount__fraction"]/text()')

# Crear un DataFrame
df = pd.DataFrame({
    'Cubierta': titulos,
    'Enlace': urls,
    'Precios': precios
}) 

print("Se creo el dataframe")

# Agregar columnas para los nuevos datos
df['Vendedor'] = ""
df['Detalles'] = ""
df['Cantidad'] = ""

print("Se agregaron las columnas para iterar")

# Iterar sobre los enlaces y obtener información adicional
for index, row in df.iterrows():
    product_url = row['Enlace']
    product_response = requests.get(product_url)
    product_soup = BeautifulSoup(product_response.content, 'html.parser')

    # Obtener el color y precio
    vendedor = product_soup.find('span', attrs={"class": "ui-pdp-color--BLUE ui-pdp-family--REGULAR"})
    detalles = product_soup.find('span', attrs={"class": "ui-pdp-subtitle"})
    cantidad = product_soup.find('span', attrs={"class":"ui-pdp-buybox__quantity__available"})
    
    # Asignar los valores a las columnas correspondientes
    df.at[index, 'Vendedor'] = vendedor.text if vendedor else "No disponible"
    df.at[index, 'Detalles'] = detalles.text if detalles else "No disponible"
    df.at[index, 'Cantidad'] = cantidad.text if cantidad else "No disponible"

print("Aguarde un momento")

# Guardar en un archivo CSV
df.to_csv('smart.csv', index=False)

print("El archivo fue exportado con exito")


404
Se creo el dataframe
Se agregaron las columnas para iterar
Aguarde un momento
El archivo fue exportado con exito


In [8]:
df

,Cubierta,Enlace,Precios,Vendedor,Detalles,Cantidad
0,Apple iPhone 12 (128 GB) - Negro - Distribuido...,https://www.mercadolibre.com.ar/apple-iphone-1...,1.319.999,No disponible,Nuevo | +100 vendidos,(10 disponibles)
1,iPhone 6s 32 GB gris espacial,https://www.mercadolibre.com.ar/iphone-6s-32-g...,329.999,MADISON_TECNO,Nuevo | +1000 vendidos,(100 disponibles)
2,Apple iPhone 14 (128 GB) - Blanco estelar,https://www.mercadolibre.com.ar/apple-iphone-1...,1.249.999,APTECNO.AR,Nuevo | +100 vendidos,No disponible
3,"Apple iPad (9ª generación) 10.2"" Wi-Fi 64GB - ...",https://www.mercadolibre.com.ar/apple-ipad-9-g...,433.800,DREAMTEC,Nuevo | +10mil vendidos,(6 disponibles)
4,Apple iPhone 15 (128 GB) - Rosa,https://www.mercadolibre.com.ar/apple-iphone-1...,1.383.171,HYT.COM.AR,Nuevo | +5 vendidos,No disponible
5,Apple iPhone 15 Pro Max (256 GB) - Titanio Azul,https://www.mercadolibre.com.ar/apple-iphone-1...,2.149.999,MOTEC_ELECTRONICA_AR,Nuevo | +25 vendidos,(5 disponibles)
6,"Apple Macbook Air (13 pulgadas, 2020, Chip M1,...",https://www.mercadolibre.com.ar/apple-macbook-...,1.389.990,ATARITECHNOLOGY,Nuevo | +5mil vendidos,No disponible
7,Apple iPhone 15 Pro (128 GB) - Titanio Azul,https://www.mercadolibre.com.ar/apple-iphone-1...,1.813.499,MOTEC_ELECTRONICA_AR,Nuevo | +5 vendidos,(5 disponibles)
8,Vidrio Templado Anti Espia Spigen Ez Para iPho...,https://click1.mercadolibre.com.ar/mclics/clic...,52.999,DRESSMYCELL,Nuevo | +25 vendidos,(8 disponibles)
9,Funda Bolsa Sumergible Celular Ringke Ipx8 50 ...,https://click1.mercadolibre.com.ar/mclics/clic...,32.999,CELUGADGETS,Nuevo | +100 vendidos,(6 disponibles)
